# Elongation of a tapered bar under axial load
![Illustraion of a tapered bar](./images/Tapered-Bar.jpg)  
given $d_1=10$ cm, $d_2=5$ cm, $P=10$ kN, $L=50$ cm, $E=200$ GPa

The force acting on the nodes 1,2 of a spring element  
$$\sigma=E\epsilon \implies \frac{f_i}{A_j}=E\frac{u_i-u_{i\pm1}}{l_o} \implies f_1 = \frac{EA_j}{l_o} (u_1-u_2),\ f_2 = \frac{EA_j}{l_o} (u_2-u_1)$$  
where $i=1, 2$ ; $j$ are the indices of nodes and elements respectively. $l_o$ is the length of the element.  

The system of equations for a spring element:  
$$\frac{EAel}{l_o} \begin{bmatrix} 1 & -1 \\ -1 & 1 \end{bmatrix} \begin{Bmatrix} u_1 \\ u_2 \end{Bmatrix} = \begin{Bmatrix} f_1 \\ f_2 \end{Bmatrix} $$

The analytical solution:  
$$u(x)=\frac{4P}{\pi E} \times \frac{xL}{Ld_1^2-(d_1-d_2)xd_1}$$

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from ipywidgets import interact
plt.rcParams['figure.figsize']=(7,7)
plt.rcParams.update({'font.size': 13})

## Input Variables

In [42]:
E = 200e3 #MPa
P = 10e3 #Newtons
d1= 10e1 #mm
d2 = 5e1 #mm
L = 50e1 #mm

In [43]:
def ana_sol(x): return (4*P/math.pi/E)*(x*L/(L*d1**2-(d1-d2)*x*d1))
u_ana = np.vectorize(ana_sol)

def get_dia(iel, xpoints):
   position = (xpoints[iel]+xpoints[iel-1])/2
   dia = ((d2-d1)/L)*(position) + d1
   return dia

def get_kf(dia, nele):
   l_o = L/nele
   # element stiffness matrix
   k = (E*(math.pi/4*dia**2)/l_o)*np.array([[1, -1], [-1, 1]])
   # element load vector
   f = np.array([[-P],[P]])
   return k, f

In [44]:
def plot_deformation(nele):
   xpoints = np.linspace(0,L,nele+1)
   # Define global stiffness matrix and global load vector
   K = np.zeros([nele+1,nele+1])
   F = np.zeros([nele+1,1])
   u = np.zeros([nele+1,1])

   # estimate global stiffness matrix and global load vector
   for iel in np.arange(1,nele+1):
      dia = get_dia(iel, xpoints)
      k,f = get_kf(dia, nele)
      K[iel-1:iel+1,iel-1:iel+1] += k
      F[iel-1:iel+1] += f

   u_sol = np.linalg.solve(K[1:,1:],F[1:])
   u[1:] = u_sol

   # Plotting the results
   xpoints_con = np.linspace(0,L,200)
   plt.plot(xpoints_con, u_ana(xpoints_con), linewidth=1, label='analytical')
   plt.plot(xpoints, u, 'o:r', mfc = 'none', label='FEM')
   plt.title("Tapered bar: displacement at 'x'")
   plt.xlabel('x')
   plt.ylabel('u')
   plt.ylim(0,8e-3)
   plt.xlim(0,510)
   plt.legend(loc='upper right')
   print("Elongation of the tapered bar is: %15.12f mm." % (u_sol[-1].item()))
   print("Anaylitical solution is: %14.12f mm." % (P*L/(math.pi/4*d1*d2*E)))

In [59]:
# Define a slider with layout options to change its physical length
slider = widgets.IntSlider(
   value=20,  # Default value
   min=1,     # Minimum value
   max=50,   # Maximum value
   step=1,    # Step size
   description='No. of Elements:',
   continuous_update=True,  # Update only after user finishes
   orientation='horizontal',  # Horizontal slider
   readout=True,  # Show the value
   readout_format='d',  # Round the output to integer
   layout=widgets.Layout(width='600px')  # Adjust the length here
)
interact(plot_deformation, nele=slider)

interactive(children=(IntSlider(value=20, description='No. of Elements:', layout=Layout(width='600px'), max=50…

<function __main__.plot_deformation(nele)>